# Fine tune StableLM-2-Zephyr-1.6B model using Axolotl framework

How to install dependencies (in HPC environment):

- load Python and cuDNN modules
- create a Python venv and activate it
- install dependencies from requirements.txt (e.g. torch)
- install Axolotl from git clone (pip won't work, see [this issue](https://github.com/OpenAccess-AI-Collective/axolotl/issues/945)):

```
git clone git@github.com:OpenAccess-AI-Collective/axolotl.git
cd axolotl
pip install -e '.[flash-attn,deepspeed]'
```


In [1]:
# Check if GPU is available
import torch
print('GPU available?', torch.cuda.is_available())
print('BF16 is supported?', torch.cuda.is_bf16_supported())

GPU available? True
BF16 is supported? True


In [2]:
# set model name etc.

MODEL_NAME = "stabilityai/stablelm-2-zephyr-1_6b"
MODEL_SHORT_NAME = MODEL_NAME.split('/')[-1]
SUFFIX = "FinGreyLit"

In [3]:
# Load and prepare fine-tuning dataset

import json
import glob
import random

random.seed(42)  # for deterministic sampling of test set

train_files = glob.glob("../../llm-dataset/*-train.jsonl")
test_files = glob.glob("../../llm-dataset/*-test.jsonl")

EVAL_SIZE = 32  # how many documents to evaluate (i.e. calculate loss) on during fine-tuning
SYSTEM_PROMPT = "You are a skilled librarian specialized in meticulous cataloguing of digital documents."
INSTRUCTION = "Extract metadata from this document. Return as JSON."

def preprocess_sample(sample):
    output = json.dumps(sample["ground_truth"])
    input_ = json.dumps(sample["content"])
    # ShareGPT format
    conversations = [
        {'from': 'system', 'value': SYSTEM_PROMPT},
        {'from': 'user', 'value': INSTRUCTION + "\n\n" + input_},
        {'from': 'gpt', 'value': output}
    ]
    return {"conversations": conversations}

def dataset_to_records(files):
    records = []
    for filename in files:
        with open(filename) as infile:
            for line in infile:
                sample = json.loads(line)
                records.append(preprocess_sample(sample))
    return records

def write_jsonl(records, filename):
    with open(filename, "w") as outfile:
        for record in records:
            json.dump(record, outfile)
            outfile.write("\n")

train_recs = dataset_to_records(train_files)
random.shuffle(train_recs)
write_jsonl(train_recs, "ax-train.jsonl")
print(f"Wrote {len(train_recs)} train records")

test_recs = dataset_to_records(test_files)
write_jsonl(test_recs, "ax-test.jsonl")
print(f"Wrote {len(test_recs)} test records")

eval_recs = random.sample(test_recs, EVAL_SIZE)
write_jsonl(eval_recs, "ax-eval.jsonl")
print(f"Wrote {len(eval_recs)} eval records")

Wrote 620 train records
Wrote 180 test records
Wrote 32 eval records


In [4]:
# Create Axolotl configuration file

CONFIG_FILE = f"config-{MODEL_SHORT_NAME}.yml"


CONFIG = f"""
base_model: {MODEL_NAME}
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
trust_remote_code: true

load_in_8bit: true
load_in_4bit: false
strict: false

datasets:
  - path: ax-train.jsonl
    ds_type: json
    split: train
    type: sharegpt
    conversation: chatml

test_datasets:
  - path: ax-eval.jsonl
    ds_type: json
    split: train
    type: sharegpt
    conversation: chatml

output_dir: ./out-{MODEL_SHORT_NAME}

chat_template: chatml

adapter: lora
lora_r: 16
lora_alpha: 32
lora_dropout: 0.05
lora_target_linear: true

sequence_len: 4096
sample_packing: false
eval_sample_packing: false
pad_to_sequence_len: true

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 4
micro_batch_size: 4
eval_batch_size: 4
num_epochs: 5
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: false

gradient_checkpointing: true  # true: saves VRAM but is slower to train
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: false  # true gives RuntimeError: FlashAttention only support fp16 and bf16 data type

warmup_steps: 10
evals_per_epoch: 2
eval_table_size:
eval_table_max_new_tokens: 128
saves_per_epoch: 1
debug:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:

""".strip()

with open(CONFIG_FILE, 'w') as outfile:
    print(CONFIG, file=outfile)

In [5]:
%%time

!venv/bin/accelerate launch -m axolotl.cli.train {CONFIG_FILE}

[2024-07-30 16:16:49,013] [INFO] [datasets.<module>:58] [PID:826120] PyTorch version 2.3.1 available.
[2024-07-30 16:17:03,330] [WARNING] [axolotl.utils.config.models.input.hint_trust_remote_code:316] [PID:826120] [RANK:0] `trust_remote_code` is set to true. Please make sure that you reviewed the remote code/model.
[2024-07-30 16:17:03,624] [INFO] [axolotl.normalize_config:183] [PID:826120] [RANK:0] GPU memory usage baseline: 0.000GB (+0.818GB misc)
                                 dP            dP   dP 
                                 88            88   88 
      .d8888b. dP.  .dP .d8888b. 88 .d8888b. d8888P 88 
      88'  `88  `8bd8'  88'  `88 88 88'  `88   88   88 
      88.  .88  .d88b.  88.  .88 88 88.  .88   88   88 
      `88888P8 dP'  `dP `88888P' dP `88888P'   dP   dP 
                                                       
                                                       

****************************************
**** Axolotl Dependency Versions *****
  accelerate: 0.3

# Use the fine-tuned model

In [6]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

qlora_model = f"./out-{MODEL_SHORT_NAME}"
base_model = MODEL_NAME
tokenizer = AutoTokenizer.from_pretrained(qlora_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map=0, trust_remote_code=True, torch_dtype="auto", attn_implementation="flash_attention_2",)
model = PeftModel.from_pretrained(model, qlora_model)



/wrk-vakka/users/oisuomin/git/FinGreyLit/experiments/axolotl-finetune-llm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# push the LoRA model (PEFT adapter) and modified tokenizer (added ChatML template) to HF Hub

hub_model_id = f"NatLibFi/{MODEL_SHORT_NAME}-{SUFFIX}"
model.push_to_hub(hub_model_id)
tokenizer.push_to_hub(hub_model_id)

adapter_model.safetensors: 100%|██████████| 30.3M/30.3M [00:02<00:00, 11.1MB/s]


CommitInfo(commit_url='https://huggingface.co/NatLibFi/stablelm-2-zephyr-1_6b-FinGreyLit/commit/8b470730559869e2f8f10f439d2bb6af1e6daedd', commit_message='Upload tokenizer', commit_description='', oid='8b470730559869e2f8f10f439d2bb6af1e6daedd', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
# merge the LoRA into the base model for inference

model = model.merge_and_unload()

In [9]:
def generate(messages):
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        #input_ids,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    output_ids = output_ids[0][len(input_ids[0]):]
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip()


In [10]:
%%time

import json

with open(f'test-records-{MODEL_SHORT_NAME}.jsonl', 'w') as outfile:
    for rec in test_recs:
        messages = [
            {"role": msg["from"], "content": msg["value"]}
            for msg in rec["conversations"]
            if msg["from"] != "gpt"
        ]
        response = generate(messages)

        ground_truth = rec['conversations'][-1]["value"]

        print(100 * "-")
        print("Ground Truth:")
        print(ground_truth)
        print("Prediction:")
        print(response)

        ground_truth = json.loads(ground_truth)

        try:
            prediction = json.loads(response)
        except json.JSONDecodeError:
            prediction = {}
        
        # rowid is set to unknown as we've lost it somewhere along the way...
        record = {"ground_truth": ground_truth, "prediction": prediction, "rowid": "unknown"}
        json.dump(record, outfile)
        outfile.write("\n")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


----------------------------------------------------------------------------------------------------
Ground Truth:
{"language": "fin", "title": "Ammattikielisten tekstien tutkimisesta - esimerkkin\u00e4 tilintarkastuskertomus", "creator": ["Katajam\u00e4ki, Heli", "Koskela, Merja"], "year": "2012", "publisher": ["Kotimaisten kielten keskus"], "type_coar": "book part"}
Prediction:
{"language": "fin", "title": "Ammattikielisten tekstien tutkimisesta : esimerkkin\u00e4 tilintarkastuskertomus", "creator": ["Koskela, Merja", "Katajam\u00e4ki, Heli"], "year": "2012", "publisher": ["Vaasan yliopisto"], "p-isbn": ["9789523969932"], "type_coar": "book part"}
----------------------------------------------------------------------------------------------------
Ground Truth:
{"language": "fin", "title": "FAQ : Taiteen digitaaliset toimintaymp\u00e4rist\u00f6t", "year": "2018", "publisher": ["Tampereen ammattikorkeakoulu"], "e-isbn": ["9789527266076"], "p-issn": "1456-002X", "type_coar": "book"}
Pre

Token indices sequence length is longer than the specified maximum sequence length for this model (4915 > 4096). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


----------------------------------------------------------------------------------------------------
Ground Truth:
{"language": "eng", "title": "Supporting Transformative Agency among Urban Actors in the Change Laboratory Intervention", "creator": ["Lund, Virpi"], "year": "2021", "publisher": ["Scientific Research Publishing"], "doi": "10.4236/cus.2021.93025", "e-issn": "2328-4919", "p-issn": "2328-4900", "type_coar": "research article"}
Prediction:
{"language": "eng", "title": "Supporting Transformative Agency among Urban Actors in the Change Laboratory Intervention", "creator": ["Lund, Virpi"], "year": "2021", "publisher": ["Scientific Research Publishing"], "doi": "10.4236/cus.2021.93025", "e-issn": "2328-4919", "type_coar": "research article"}
----------------------------------------------------------------------------------------------------
Ground Truth:
{"language": "eng", "title": "The roles of temperature, nest predators and information parasites for geographical variation in 

In [11]:
# Analyze the statistics of the extracted metadata and save to file

import sys
sys.path.append('..')

from eval import MetadataEvaluator

evaluator = MetadataEvaluator(f'test-records-{MODEL_SHORT_NAME}.jsonl')
results = evaluator.evaluate_records() #prediction_records[:9])

statistics_filename = '../results-axolotl-' + MODEL_SHORT_NAME + '.md'
evaluator.save_md(results, statistics_filename)

In [12]:
# Save the merged model to a directory (along with tokenizer)

merged_model_dir = f"merged-{MODEL_SHORT_NAME}"
model.save_pretrained(merged_model_dir)
tokenizer.save_pretrained(merged_model_dir)

('merged-stablelm-2-zephyr-1_6b/tokenizer_config.json',
 'merged-stablelm-2-zephyr-1_6b/special_tokens_map.json',
 'merged-stablelm-2-zephyr-1_6b/vocab.json',
 'merged-stablelm-2-zephyr-1_6b/merges.txt',
 'merged-stablelm-2-zephyr-1_6b/added_tokens.json',
 'merged-stablelm-2-zephyr-1_6b/tokenizer.json')

In [4]:
%%time
# convert the merged model to GGUF using llama.cpp tools (installed separately)

LLAMA_CPP_PATH = "../../../llama.cpp"
merged_model_dir = f"merged-{MODEL_SHORT_NAME}"

!{LLAMA_CPP_PATH}/venv/bin/python {LLAMA_CPP_PATH}/convert_hf_to_gguf.py {merged_model_dir} --outfile {MODEL_SHORT_NAME}-{SUFFIX}-f16.gguf

INFO:hf-to-gguf:Loading model: merged-stablelm-2-zephyr-1_6b
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:output.weight,             torch.float16 --> F16, shape = {2048, 100352}
INFO:hf-to-gguf:token_embd.weight,         torch.float16 --> F16, shape = {2048, 100352}
INFO:hf-to-gguf:blk.0.attn_norm.bias,      torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.float16 --> F16, shape = {5632, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.float16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.float16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_norm.bias,       torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.float16 --> F32, shape = {2048}
INFO:hf-to-

In [5]:
%%time
# Quantize the F16 GGUF model to the 4+ bit Q4_K_M format
QTYPE = "Q4_K_M"

!{LLAMA_CPP_PATH}/llama-quantize {MODEL_SHORT_NAME}-{SUFFIX}-f16.gguf {MODEL_SHORT_NAME}-{SUFFIX}-{QTYPE}.gguf {QTYPE}

main: build = 3492 (7c27a19b)
main: built with cc (GCC) 13.3.0 for x86_64-pc-linux-gnu
main: quantizing 'stablelm-2-zephyr-1_6b-FinGreyLit-f16.gguf' to 'stablelm-2-zephyr-1_6b-FinGreyLit-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 27 key-value pairs and 340 tensors from stablelm-2-zephyr-1_6b-FinGreyLit-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = stablelm
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Stablelm 2 Zephyr 1_6b
llama_model_loader: - kv   3:                       general.organization str              = Stabilityai
llama_model_loader: - kv   4:                           general.basename str              = stablelm-2-zephyr
llama_model_

In [ ]:
# Upload the quantized GGUF file to HF Hub

FINAL_MODEL_NAME = f"NatLibFi/{MODEL_SHORT_NAME}-{SUFFIX}-GGUF"

!{LLAMA_CPP_PATH}/venv/bin/huggingface-cli upload {FINAL_MODEL_NAME} {MODEL_SHORT_NAME}-{SUFFIX}-{QTYPE}.gguf

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
stablelm-2-zephyr-1_6b-FinGreyLit-Q4_K_M.gguf:  99%|▉| 1.02G/1.03G [00:36<00:00,